In [5]:
:dep graph-ds = { path = "." }

In [6]:
use graph_ds::{Graph, Node, Edge};
use graph_ds::hexagon_graph::{hexagon_graph_from_file, cell::Cell};
use std::time::Instant;

In [7]:
let mut graph : Graph<Cell> = hexagon_graph_from_file("de_inferno_hexagons.mpk")?;

In [8]:
let start = Cell {
    a: 60,
    b: -33,
    radius: 24,
    layer: 3,
};
let end = Cell {
    a: 5,
    b: 61,
    radius: 24,
    layer: 3,
};

In [9]:
fn heuristic(start_cell: &Cell, end_cell: &Cell) -> f64 {
    let dx = (start_cell.a - end_cell.a).abs();
    let dy = (start_cell.b - end_cell.b).abs();
    let dz = (start_cell.a + start_cell.b - end_cell.a - end_cell.b).abs();
    let dlayer = (start_cell.layer - end_cell.layer).abs();
    ((dx + dy + dz) as f64 / 2.0 + dlayer as f64) * (start_cell.radius * 2) as f64
}

In [22]:
println!("BFS start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (path, _) = graph.bfs(start, Some(end))?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", path.unwrap().len());

println!("---");

println!("AStar start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (_, distance) = graph.astar(start, end, heuristic)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", distance);

BFS start: Cell { a: 60, b: -33, radius: 24, layer: 3 }, end: Cell { a: 5, b: 61, radius: 24, layer: 3 }
time: 602 µs
distance: 117
---
AStar start: Cell { a: 60, b: -33, radius: 24, layer: 3 }, end: Cell { a: 5, b: 61, radius: 24, layer: 3 }
time: 2111 µs
distance: 117.0
